In [1]:
pip install crewai

  Using cached crewai-0.83.0-py3-none-any.whl.metadata (19 kB)
  Using cached auth0_python-4.7.2-py3-none-any.whl.metadata (8.9 kB)
  Using cached chromadb-0.5.20-py3-none-any.whl.metadata (6.8 kB)
  Using cached crewai_tools-0.14.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached json_repair-0.30.2-py3-none-any.whl.metadata (11 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.28.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.0 MB/s eta 0:00:00
  Using cached tomli_w-1.1.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached cryptography-43.0.3-cp39-abi3-macosx_10_9_universal2.whl.metadata (5.4 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ..

In [2]:
from crewai import Agent
from crewai import LLM

In [20]:
from langchain_community.llms import Ollama

In [297]:
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Ollama
langchain_llm = Ollama(model="llama3.2")

In [23]:
import ollama

response = ollama.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. This effect occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere.

Here's what happens:

1. Sunlight enters the Earth's atmosphere and is made up of different wavelengths (colors) of light.
2. The shorter wavelengths of light (such as blue and violet) are scattered more than the longer wavelengths (like red and orange) by the tiny molecules of gases like nitrogen and oxygen in the air.
3. As a result, the shorter wavelengths of light are dispersed in all directions and reach our eyes from every part of the sky.
4. This scattering effect is more pronounced for blue light because it has a shorter wavelength, making it more likely to be scattered.

Now, let's take this to an even more specific level of explanation. When sunlight passes through the atmosphere, different parts of the sky absorb and scatter light differently due to var

In [28]:
model = Ollama(model = "llama3.2")
classifier = Agent(
    role = "email classifier",
    goal = "Accurately classify emails based on their importance. Give every email one of these ratings: important, casual, or spam.",
    backstory="You are an AI assistance whose only job "
              "is to classify emails accurately and honestly. "
              "Do not be afraid to give emails bad ratings "
              "if they are not important. "
              "Your job is to help the user manage their inbox.",
    llm = langchain_llm
)


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [80]:
from crewai import Agent, Crew, Task
from crewai import LLM

# Create an LLM instance for Ollama LLaMA 3.2
llama_model = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"  # URL for Ollama's local server
)

# Define the agent with the LLaMA model, role, backstory, and goal
agent = Agent(
    llm=llama_model,
    role="researcher",  # Define the role of the agent (e.g., 'researcher')
    backstory="I am a research assistant that helps gather and summarize information on various topics.",  # Backstory
    goal="Assist with research by answering questions and providing relevant information from various sources."  # Goal
)

# Create a Task instance for the agent to execute
task = Task(
    description="Research the latest advancements in AI technology.",  # Describe the task
    input_value={"prompt": "What are the latest advancements in AI research?"},  # Your query as input
    expected_output="A detailed response about the latest advancements in AI research."  # Expected output (optional)
)


# Interact with the agent by passing the task to the agent's `execute_task` method
response = agent.execute_task(task)

# Print the response
print("Agent Response:", response)


Agent Response: The field of Artificial Intelligence (AI) has witnessed significant advancements in recent years, with researchers exploring new techniques and applications that have improved its capabilities. Some of the latest developments in AI technology include:

1. **Advances in Deep Learning**: Deep learning techniques such as Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), and Generative Adversarial Networks (GANs) continue to improve, enabling AI systems to learn complex patterns and relationships from large datasets.
2. **Transformers**: The introduction of transformers, a type of neural network architecture, has revolutionized the field of natural language processing (NLP). Transformers have been used in various applications such as machine translation, text summarization, and question answering.
3. **Explainability and Transparency**: Researchers are working on developing more explainable AI systems that can provide insights into their decision-makin

In [261]:
import requests
from crewai import Agent, Task, LLM
from bs4 import BeautifulSoup

# Define the WebCrawlerAgent class
class WebCrawlerAgent(Agent):
    def __init__(self, llm, role, backstory, goal, serper_api_key):
        # Initialize the base Agent
        super().__init__(
            llm=llama_model,
            role=role,
            backstory=backstory,
            goal=goal
        )
        # Custom field for Serper API key
        self._serper_api_key = serper_api_key  # Keep it private with _ prefix

    def web_crawl(self, query):
        """Fetch search results from the Serper.dev API."""
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self._serper_api_key}
        payload = {"q": query}
        
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            results = response.json()
            return results.get("organic", [])  # Extract organic search results
        else:
            raise Exception(f"Serper API error: {response.status_code}, {response.text}")

    def execute_task(self, task: Task, context:dict = None, tools:str = None):
        """Execute the given task by performing a web search."""
        # Access the query from task's description or other field
        query = task.description  # Using description for query

        if not query:
            raise ValueError("Task description must include a 'query' field.")
        
        # Perform the web crawl
        search_results = self.web_crawl(query)
        print(search_results)
        return search_results#{"search_results": search_results}


# Initialize LLM (Ollama's LLaMA 3.2)
llama_model = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"  # URL for Ollama's local server
)

# Create an instance of the WebCrawlerAgent
web_crawler_agent = WebCrawlerAgent(
    llm=llama_model,
    role="web crawler",
    backstory="I search the web to gather information on a given topic.",
    goal="Fetch web search results for a given query.",
    serper_api_key="04f7de9cfae51d40e94f1e328d1aa756ffbb14ff"  # Replace with your actual Serper API key
)

# Define a task to test the agent
query = "Recent advancements in artificial intelligence"
web_task = Task(
    description=query,  # Store the query here as description
    expected_output="A detailed response about the latest advancements in AI research.",
    agent= web_crawler_agent
)

# Execute the task and print the results
try:
    articles = web_crawler_agent.execute_task(web_task)
    print("Search Results:", articles)
except Exception as e:
    print(f"Error: {e}")


[{'title': 'Advancements in Artificial Intelligence and Machine Learning', 'link': 'https://online-engineering.case.edu/blog/advancements-in-artificial-intelligence-and-machine-learning', 'snippet': 'This article will explore the latest advancements in artificial intelligence and machine learning, including recent development of advanced algorithms.', 'date': 'Mar 25, 2024', 'position': 1}, {'title': '7 Recent AI Developments: Artificial Intelligence News - Koombea', 'link': 'https://www.koombea.com/blog/7-recent-ai-developments/', 'snippet': 'AI Robots Learning Through Observation · AI Robot Caregivers Are Filling a Shortfall · AI Beer Brewers · AI-Based Cybersecurity · AI Diagnostics for ...', 'date': 'Oct 22, 2024', 'position': 2}, {'title': 'SQ2. What are the most important advances in AI?', 'link': 'https://ai100.stanford.edu/gathering-strength-gathering-storms-one-hundred-year-study-artificial-intelligence-ai100-2021-1/sq2', 'snippet': 'The field of AI has made major progress in 

In [262]:
crew = Crew(
    agents=[web_crawler_agent],
    tasks=[web_task],
    verbose=True
)

# Execute tasks in sequence
crew.kickoff()

[{'title': 'Advancements in Artificial Intelligence and Machine Learning', 'link': 'https://online-engineering.case.edu/blog/advancements-in-artificial-intelligence-and-machine-learning', 'snippet': 'This article will explore the latest advancements in artificial intelligence and machine learning, including recent development of advanced algorithms.', 'date': 'Mar 25, 2024', 'position': 1}, {'title': '7 Recent AI Developments: Artificial Intelligence News - Koombea', 'link': 'https://www.koombea.com/blog/7-recent-ai-developments/', 'snippet': 'AI Robots Learning Through Observation · AI Robot Caregivers Are Filling a Shortfall · AI Beer Brewers · AI-Based Cybersecurity · AI Diagnostics for ...', 'date': 'Oct 22, 2024', 'position': 2}, {'title': 'SQ2. What are the most important advances in AI?', 'link': 'https://ai100.stanford.edu/gathering-strength-gathering-storms-one-hundred-year-study-artificial-intelligence-ai100-2021-1/sq2', 'snippet': 'The field of AI has made major progress in 

ValidationError: 1 validation error for TaskOutput
raw
  Input should be a valid string [type=string_type, input_value=[{'title': 'Advancements ... 2024', 'position': 10}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

In [86]:

# Create a ContentSummarizerAgent class
class ContentSummarizerAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="content summarizer",
            backstory="I am an agent that summarizes research content.",
            goal="Summarize research papers and articles to provide concise insights."
        )

    def execute_task(self, task: Task):
        """Execute the summarization task."""
        # Use 'context' which contains articles for summarization
        articles = task.context  # Articles are passed in 'context' now, not in 'input_value'
        
        summaries = []
        for article in articles:
            summary = self.summarize(article["content"])
            summaries.append(summary)
        
        return {"summaries": summaries}

    def summarize(self, content):
        """Summarize the content using LLM."""
        summarize_task = Task(
            description="Summarize research content.",
            context=[{
                "description": "Summarize this research article's content.",
                "expected_output": "A concise summary of the research content.",
                "content": content  # Include the article content here
            }],
            expected_output="A concise summary of the content."  # Define the expected output
        )
        
        # Execute the summarization task with LLM
        response = self.llm.execute_task(summarize_task)
        return response["output"]  # Return the summarized content

    def fetch_article_content(self, url):
        """Fetch the article content from the URL."""
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Assuming the article's main content is in a <div> with class "article-content"
                content = soup.find('div', {'class': 'article-content'})  # Adjust based on actual HTML structure
                return content.get_text(strip=True) if content else "Content not found"
            else:
                return "Error fetching content"
        except requests.RequestException as e:
            return f"Error fetching content: {e}"



# Initialize ContentSummarizerAgent
summarizer_agent = ContentSummarizerAgent(llama_model)

# Prepare the task with the article content
search_results=[]
for article in articles:
    content = summarizer_agent.fetch_article_content(article["link"])  # Fetch content from the link
    search_results.append({
        "description": f"Summarize the article: {article['title']}",
        "expected_output": "A concise summary of the article.",
        "content": content  # Use the fetched content
    })

# Create a task to summarize the articles
summarize_task = Task(
    description="Summarize research papers.",
    context=search_results,  # Pass the structured articles as context
    expected_output="A detailed summary of research articles."  # Specify expected output for the task
)

# Execute the summarization task
response = summarizer_agent.execute_task(summarize_task)

# Print the summaries
summaries = response.get("summaries", [])
for summary in summaries:
    print(summary)


TypeError: string indices must be integers, not 'str'

In [157]:
class WebCrawlerAgent(Agent):
    def __init__(self, llm, api_key):
        super().__init__(
            llm=llm,
            role="web scraper",
            backstory="I search the web to gather information on a given topic.",
            goal="Fetch web search results for a given query."
        )
        self._api_key = api_key

    def web_crawl(self, query):
        """Fetch search results from the Serper.dev API."""
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self._api_key}
        payload = {"q": query}
        
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            results = response.json()
            return results.get("organic", [])  # Extract organic search results
        else:
            raise Exception(f"Serper API error: {response.status_code}, {response.text}")

    def execute_task(self, task: Task):
        """Execute the given task by performing a web search."""
        # Access the query from task's description or other field
        query = task.description  # Using description for query

        if not query:
            raise ValueError("Task description must include a 'query' field.")
        
        # Perform the web crawl
        search_results = self.web_crawl(query)
        return {"search_results": search_results}

In [166]:
import requests
from crewai import Agent, Task

class WebScraperAgent(Agent):
    def __init__(self, llm, api_key):
        super().__init__(
            llm=llm,
            role="web scraper",
            backstory="I search the web to gather information on a given topic.",
            goal="Fetch web search results for a given query."
        )
        self._api_key = api_key

    def web_crawl(self, query):
        """Fetch search results from the Serper.dev API."""
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self._api_key}
        payload = {"q": query}
        
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            return response.json().get("organic", [])  # Extract organic search results
        else:
            raise Exception(f"Serper API error: {response.status_code}, {response.text}")

    def execute_task(self, task: Task):
        """Execute the task by performing a web search."""
        query = task.description  # Task description contains the search query
        
        if not query:
            raise ValueError("Task description must include a search query.")
        
        # Perform the web crawl
        search_results = self.web_crawl(query)
        return {"search_results": search_results}


In [142]:
class SummarizerAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="content summarizer",
            backstory="I summarize research content into concise insights.",
            goal="Summarize research articles or content to provide clear and precise information."
        )

    def execute_task(self, task: Task):
        """Execute the summarization task."""
        if not task.context or "search_results" not in task.context:
            raise ValueError("Summarization task requires 'search_results' in context.")

        search_results = task.context["search_results"]
        summaries = []
        for result in search_results:
            content = result.get("snippet", "")  # Use snippet for summarization
            summary = self.summarize(content)
            summaries.append({"title": result.get("title", ""), "summary": summary})

        return {"summaries": summaries}

    def summarize(self, content):
        """Summarize the content using the LLM."""
        task = Task(
            description="Summarize content.",
            context={"content": content},
            expected_output="A concise summary of the content."
        )
        response = self.llm.execute_task(task)
        return response["output"]


In [143]:
class BiasDetectionAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="bias detector",
            backstory="I analyze research summaries for bias.",
            goal="Identify and flag any potential biases in summaries."
        )

    def execute_task(self, task: Task):
        """Execute the bias detection task."""
        if not task.context or "summaries" not in task.context:
            raise ValueError("Bias detection task requires 'summaries' in context.")

        summaries = task.context["summaries"]
        bias_results = []
        for summary in summaries:
            analysis = self.detect_bias(summary["summary"])
            bias_results.append({"title": summary["title"], "bias_analysis": analysis})

        return {"bias_results": bias_results}

    def detect_bias(self, content):
        """Analyze content for bias using the LLM."""
        task = Task(
            description="Analyze content for bias.",
            context={"content": content},
            expected_output="A detailed analysis of any bias in the content."
        )
        response = self.llm.execute_task(task)
        return response["output"]


In [144]:
class ReportGeneratorAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="report generator",
            backstory="I compile summaries and bias analyses into a comprehensive report.",
            goal="Generate a final research report based on the processed data."
        )

    def execute_task(self, task: Task):
        """Execute the report generation task."""
        if not task.context or "bias_results" not in task.context:
            raise ValueError("Report generation task requires 'bias_results' in context.")

        bias_results = task.context["bias_results"]
        report = self.generate_report(bias_results)
        return {"report": report}

    def generate_report(self, bias_results):
        """Generate a report using the LLM."""
        task = Task(
            description="Generate a research report.",
            context={"bias_results": bias_results},
            expected_output="A detailed research report based on summaries and bias analyses."
        )
        response = self.llm.execute_task(task)
        return response["output"]


In [167]:
web_scraper = WebScraperAgent(llm=llama_model, api_key="04f7de9cfae51d40e94f1e328d1aa756ffbb14ff")
summarizer = SummarizerAgent(llm=llama_model)
bias_detector = BiasDetectionAgent(llm=llama_model)
report_generator = ReportGeneratorAgent(llm=llama_model)

ValidationError: 3 validation errors for WebScraperAgent
role
  Field required [type=missing, input_value={'llm': <crewai.llm.LLM object at 0x34945cc90>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
goal
  Field required [type=missing, input_value={'llm': <crewai.llm.LLM object at 0x34945cc90>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
backstory
  Field required [type=missing, input_value={'llm': <crewai.llm.LLM object at 0x34945cc90>}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [165]:
web_scraper_task = Task(
    description="Latest advancements in AI",  # Web scraping task
    expected_output="Search results with URLs and snippets.",
    agent=web_scraper  # Explicitly associate the task with the WebScraperAgent
)

summarizer_task = Task(
    description="Summarize the web data.",  # Summarizing task
    expected_output="Concise summaries of the search results.",
    agent=summarizer  # Explicitly associate the task with the SummarizerAgent
)

bias_detection_task = Task(
    description="Detect bias in the summaries.",  # Bias detection task
    expected_output="Bias analysis of the summaries.",
    agent=bias_detector  # Explicitly associate the task with the BiasDetectionAgent
)

report_task = Task(
    description="Generate a final research report.",  # Report generation task
    expected_output="A comprehensive research report.",
    agent=report_generator  # Explicitly associate the task with the ReportGeneratorAgent
)

# Create the crew and start the process
crew = Crew(
    agents=[web_scraper, summarizer, bias_detector, report_generator],
    tasks=[web_scraper_task, summarizer_task, bias_detection_task, report_task]
)

# Execute the tasks in sequence
crew.kickoff()

TypeError: WebScraperAgent.execute_task() got an unexpected keyword argument 'context'

In [181]:
import requests
from crewai import Agent, Task, LLM
from bs4 import BeautifulSoup

# WebCrawlerAgent: Scrapes the web for search results
class WebCrawlerAgent(Agent):
    def __init__(self, llm, role, backstory, goal, serper_api_key):
        super().__init__(llm=llm, role=role, backstory=backstory, goal=goal)
        self._serper_api_key = serper_api_key

    def web_crawl(self, query):
        """Fetch search results from the Serper.dev API."""
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self._serper_api_key}
        payload = {"q": query}
        
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            results = response.json()
            return results.get("organic", [])
        else:
            raise Exception(f"Serper API error: {response.status_code}, {response.text}")

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
            """Execute the task by performing a web search and returning results as a string."""
            query = task.description
            if not query:
                raise ValueError("Task description must include a 'query' field.")
            
            # Perform the web crawl
            search_results = self.web_crawl(query)
            
            # Format the search results as a string (e.g., by joining titles)
            search_results_str = "\n".join([result['title'] for result in search_results if 'title' in result])
            
            # Ensure we return a string as expected by TaskOutput
            return {"raw": search_results_str}  # 'raw' should be a string



In [170]:
class SummarizerAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="summarizer",
            backstory="I summarize the data gathered from the web.",
            goal="Generate concise summaries from raw web data."
        )

    def execute_task(self, task: Task):
        """Summarize the search results."""
        search_results = task.description  # Get the search results from the task
        summaries = [self.summarize(result) for result in search_results]
        return {"summaries": summaries}

    def summarize(self, article):
        """Summarize an individual article."""
        return self.llm.execute_task(Task(description=f"Summarize the article: {article['title']}"))


class BiasDetectionAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="bias detector",
            backstory="I analyze the summaries for any potential bias.",
            goal="Detect and report any bias present in the summaries."
        )

    def execute_task(self, task: Task):
        """Detect bias in the summaries."""
        summaries = task.description  # Get the summaries to check for bias
        bias_analysis = [self.detect_bias(summary) for summary in summaries]
        return {"bias_analysis": bias_analysis}

    def detect_bias(self, summary):
        """Analyze the content for bias."""
        return self.llm.execute_task(Task(description=f"Detect bias in: {summary}"))


class ReportGeneratorAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="report generator",
            backstory="I generate a final comprehensive report from the summaries and bias analysis.",
            goal="Create a detailed report summarizing the research findings."
        )

    def execute_task(self, task: Task):
        """Generate a comprehensive report."""
        summaries = task.description  # Get the summaries and bias analysis
        report = self.generate_report(summaries)
        return {"report": report}

    def generate_report(self, summaries):
        """Generate a comprehensive research report."""
        return self.llm.execute_task(Task(description=f"Generate a report from: {summaries}"))

In [182]:
# Initialize LLM (Ollama's LLaMA 3.2)
llama_model = LLM(model="ollama/llama3.2", base_url="http://localhost:11434")

# Initialize agents
web_crawler_agent = WebCrawlerAgent(
    llm=llama_model,
    role="web crawler",
    backstory="I search the web to gather information on a given topic.",
    goal="Fetch web search results for a given query.",
    serper_api_key="04f7de9cfae51d40e94f1e328d1aa756ffbb14ff"
)

summarizer_agent = SummarizerAgent(llm=llama_model)
summarizer_task = Task(description="Summarize the web data.", expected_output="Concise summaries of the search results.", agent=summarizer_agent)
bias_detector_agent = BiasDetectionAgent(llm=llama_model)
report_generator_agent = ReportGeneratorAgent(llm=llama_model)

# Create tasks for each agent
query = "Recent advancements in artificial intelligence"
web_scraper_task = Task(description=query, expected_output="Search results with URLs and snippets.", agent=web_crawler_agent)
summarizer_task = Task(description="Summarize the web data.", expected_output="Concise summaries of the search results.", agent=summarizer_agent)
bias_detection_task = Task(description="Detect bias in the summaries.", expected_output="Bias analysis of the summaries.", agent=bias_detector_agent)
report_task = Task(description="Generate a final research report.", expected_output="A comprehensive research report.", agent=report_generator_agent)

# Create the Crew with all agents and tasks
crew = Crew(
    agents=[web_crawler_agent],
    tasks=[web_scraper_task]
)

# Execute tasks in sequence
crew.kickoff()



ValidationError: 1 validation error for TaskOutput
raw
  Input should be a valid string [type=string_type, input_value={'raw': "Advancements in ...elligence Trends - IBM"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

# Starts here 

In [243]:

# Define a task to test the agent
query = "Recent advancements in artificial intelligence"
web_crawler_task = Task(
    description=query,  # Store the query here as description
    expected_output="A detailed response about the latest advancements in AI research."
)

# Execute the task and print the results
try:
    articles = web_crawler_agent.execute_task(web_crawler_task)
    print("Search Results:", articles["search_results"])
except Exception as e:
    print(f"Error: {e}")

Error: string indices must be integers, not 'str'


"Advancements in Artificial Intelligence and Machine Learning\n7 Recent AI Developments: Artificial Intelligence News - Koombea\nSQ2. What are the most important advances in AI?\nLatest Development of Artificial Intelligence | InData Labs\n11 New Technologies in AI: All Trends of 2023-2024 - devabit\nArtificial Intelligence News - ScienceDaily\nArtificial Intelligence's Use and Rapid Growth Highlight Its ... - GAO\n10 top AI and machine learning trends for 2024 | TechTarget\nThe Top Artificial Intelligence Trends - IBM\nTech companies unveil rapid AI advancements, sparking wonder ..."

In [288]:
class WebCrawlerAgent(Agent):
    def __init__(self, llm, role, backstory, goal, serper_api_key):
        super().__init__(llm=llm, role=role, backstory=backstory, goal=goal)
        self._serper_api_key = serper_api_key

    def web_crawl(self, query):
        """Fetch search results from the Serper.dev API."""
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self._serper_api_key}
        payload = {"q": query}
        
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            try:
                results = response.json()
                return results.get("organic", [])
            except Exception as e:
                raise ValueError(f"Failed to parse JSON: {e}")
        else:
            raise Exception(f"Serper API error: {response.status_code}, {response.text}")

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
        """Execute the task by performing a web search and returning results as a string."""
        query = task.description
        if not query:
            raise ValueError("Task description must include a 'query' field.")
        
        # Perform the web crawl
        search_results = self.web_crawl(query)
        
        # Format the search results as a string (e.g., by joining titles)
        search_results_str = "\n".join(
            [result.get('title', 'No Title') for result in search_results]
        )
        
        # Ensure we return a string as expected by TaskOutput
        return search_results_str

In [322]:
class SummarizerAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="Content Summarizer",
            backstory="I summarize research content into concise insights.",
            goal="Summarize research articles or content to provide clear and precise information."
        )

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
        """Execute the summarization task."""
        if not context:
            raise ValueError("Summarization task requires content in context.")
        
        # Raw content might be a string or a list of strings
        raw_content = context
        if isinstance(raw_content, str):
            summary = self.summarize(raw_content)
        elif isinstance(raw_content, list) and all(isinstance(item, str) for item in raw_content):
            summary = self.summarize("\n".join(raw_content))  # Combine all strings into one for summarization
        else:
            raise ValueError("Raw content must be a string or a list of strings.")
        
        return summary

    def summarize(self, content):
        """Summarize the content using the LLM."""
        # Directly summarize the content using LLM's capabilities
        #return self.llm(content)
        return self.llm.call([
            {"role": "user", "content": content}
        ])

In [323]:
# Define the query and task
query = "Recent advancements in artificial intelligence"


# Initialize the agents
web_crawler_agent = WebCrawlerAgent(
    llm=llama_model,
    role="web crawler",
    backstory="I search the web to gather information on a given topic.",
    goal="Fetch web search results for a given query.",
    serper_api_key="04f7de9cfae51d40e94f1e328d1aa756ffbb14ff"
)

web_crawler_task = Task(
    description=query,  # Store the query here as description
    expected_output="A detailed response about the latest advancements in AI research.",
    agent = web_crawler_agent
)



summarizer_agent = SummarizerAgent(llama_model)
summarizer_task = Task(description="Summarize the web data.", expected_output="Concise summaries of the search results.", agent=summarizer_agent)



# Create the crew with all agents
crew = Crew(
    agents=[web_crawler_agent, summarizer_agent],
    tasks=[web_crawler_task,summarizer_task],
    verbose=True,

    process=Process.sequential
)

# Execute tasks in sequence
output1=crew.kickoff()


In [325]:
output1.raw

'Based on the list of articles, some of the most important advances in AI include:\n\n1. **Increased use of AI in various industries**: The growth of AI is being highlighted in multiple articles, indicating a widespread adoption of AI technologies across different sectors.\n2. **Advances in machine learning**: Several articles mention machine learning as a key area of development, suggesting significant progress in the field.\n3. **Emergence of new AI technologies**: Articles like "11 New Technologies in AI" and "The Top Artificial Intelligence Trends" suggest the discovery of new AI-related technologies that are gaining traction.\n4. **Improved AI applications**: The articles mention various applications of AI, such as healthcare (e.g., GAO\'s report), finance (e.g., IBM\'s article), and education (e.g., "The Evolution and Future of Artificial Intelligence: A Student\'s Guide").\n\nSome specific developments mentioned in the articles include:\n\n1. **Increased use of AI in healthcare*

In [299]:

langchain_llm.invoke(output1.raw)   

'Based on the provided sources, here are some of the most important advances in AI:\n\n1. **Advances in Explainable AI (XAI)**: XAI is a subfield of AI that focuses on making machine learning models more transparent and interpretable. This allows users to understand how models make decisions and reduces bias.\n2. **Rise of Edge AI**: Edge AI refers to the deployment of AI models on edge devices, such as smartphones, smart home devices, and cars. This enables real-time processing and reduces latency.\n3. **Increased Adoption of Reinforcement Learning**: Reinforcement learning is a type of machine learning that involves training models through trial and error. Its adoption has increased in recent years, leading to advancements in areas like robotics and game playing.\n4. **Advances in Natural Language Processing (NLP)**: NLP has made significant progress in recent years, enabling machines to understand and generate human language more accurately. This has led to applications in chatbots,

In [292]:
output1

CrewOutput(raw="Advancements in Artificial Intelligence and Machine Learning\n7 Recent AI Developments: Artificial Intelligence News - Koombea\nSQ2. What are the most important advances in AI?\nLatest Development of Artificial Intelligence | InData Labs\n11 New Technologies in AI: All Trends of 2023-2024 - devabit\nArtificial Intelligence News - ScienceDaily\nThe Evolution and Future of Artificial Intelligence: A Student's Guide\nArtificial Intelligence's Use and Rapid Growth Highlight Its ... - GAO\n10 top AI and machine learning trends for 2024 | TechTarget\nThe Top Artificial Intelligence Trends - IBM", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Recent advancements in artificial intelligence', name=None, expected_output='A detailed response about the latest advancements in AI research.', summary='Recent advancements in artificial intelligence...', raw="Advancements in Artificial Intelligence and Machine Learning\n7 Recent AI Developments: Artificial Intelli

In [280]:
crew_result = crew.kickoff(inputs={'context': query})
print(crew_result)

TypeError: 'LLM' object is not callable

In [188]:
langchain_llm.generate

In [286]:
ticket = "Customer support ticket: Unable to access account"

# Define the Ticket Classifier Agent
# This defines the ticket classifier agent. It's responsible for classifying incoming customer support tickets into relevant categories.
classifier = Agent(
    role="Ticket Classifier",
    goal="Accurately classify incoming customer support tickets into relevant categories: urgent, routine, or non-support-related.",
    backstory="You are an AI assistant tasked with classifying incoming customer support tickets to help streamline the support process.",
    verbose=True,
    allow_delegation=False,
    llm= llama_model
)

# Define the Ticket Action Agent
# This defines the ticket action taker agent. It's responsible for taking appropriate actions based on the classification provided by the classifier agent.
action_taker = Agent(
    role="Ticket Action Taker",
    goal="Take appropriate actions based on the classification of the customer support ticket: notify appropriate personnel for urgent issues, assign routine inquiries to relevant support teams, and filter out non-support-related messages.",
    backstory="You are an AI assistant responsible for taking actions on classified customer support tickets to ensure timely resolution.",
    verbose=True,
    allow_delegation=False,
    llm= llama_model
)



classify_ticket = Task(
    description=f"Classify the following support ticket: '{ticket}'",
    agent=classifier,
    expected_output="One of these three options: 'urgent', 'routine', or 'non-support-related'"
)

# This task is to take appropriate action on the support ticket based on the classification provided by the classifier agent.
take_action_on_ticket = Task(
    description=f"Take appropriate action on the support ticket: '{ticket}' based on the classification provided by the 'classifier' agent.",
    agent=action_taker,
    expected_output="An action taken on the ticket based on the classification provided by the 'classifier' agent."
)
from crewai import Agent, Task, Crew, Process
crew = Crew(
    agents=[classifier, action_taker],
    tasks=[classify_ticket, take_action_on_ticket],
    verbose=True,
    process=Process.sequential
)

output = crew.kickoff()
print(output)

# Agent: Ticket Classifier
## Task: Classify the following support ticket: 'Customer support ticket: Unable to access account'


# Agent: Ticket Classifier
## Final Answer: 
The support ticket "Unable to access account" can be classified as 'routine'.


# Agent: Ticket Action Taker
## Task: Take appropriate action on the support ticket: 'Customer support ticket: Unable to access account' based on the classification provided by the 'classifier' agent.


# Agent: Ticket Action Taker
## Final Answer: 
Based on the classification provided by the classifier agent, I will assign this routine inquiry to the relevant support team for further assistance. The support team will review the account access issue and provide a resolution or escalate the issue if necessary. In this case, since the ticket is classified as 'routine', it can be assigned to our standard account troubleshooting team for review and potential solution.

The action taken on the ticket includes:

* Creating a new ticket in our